In [3]:
import os
import tqdm
import string
import random
import matplotlib.pyplot as plt
import numpy
import cv2
import tensorflow as tf

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

2023-02-15 16:24:30.375254: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
CLASSES = {
    '1' : 2,  # VEHICLE
    '2' : 0,  # PEDESTRIAN
    '3' : 9,  # SIGN
    '4' : 1,  # CYCLIST
}



def create_unique_file_name():
    """ Create a .txt file with random name 
        
        Return
        ------
        String
    """
    return ''.join([random.choice(string.ascii_letters[0 : 26] + string.digits) for n in range(16)]) + '-waymo'


def create_txt_file(EXTRACT_PATH, FILE_NAME):
    """ Create the .txt file with unique name
    
    Parameters
    -----------
    EXTRACT_PATH - relative folder path
    FILE_NAME - generated file name
    
    Return
    -----------
    None
    
    """
    open(f'{EXTRACT_PATH}{FILE_NAME}.txt', 'w')
    return


def normalize(class_id, center_x, center_y, box_width, box_height, image_size):
    img_h, img_w, _ = image_size    
    
    center_x /= img_w
    center_y /= img_h
    box_width /= img_w
    box_height /= img_h
    
    return f'{class_id} {center_x} {center_y} {box_width} {box_height}'
    


def save_image(current_frame, camera_labels, EXTRACT_PATH, UNIQUE_NAME):
    """ Saves the image into given folder
    
    Parameters
    ----------
    current_frame - current frame
    EXTRACT_PATH - folder path for saving the image
    UNIQUE_NAME - random generated image name
    
    
    Return
    ----------
    Image in RGB format
    
    """
    for camera_image in current_frame.images:
        if camera_image.name == camera_labels.name:
            img = tf.image.decode_jpeg(camera_image.image).numpy()
            img_gbr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            cv2.imwrite(f'{EXTRACT_PATH}{UNIQUE_NAME}.jpg', img_gbr)

            return img_gbr

def get_image_size(image):
    """ Gets the image size
    
    Parameters
    -----------
    image - current image in RGB format
    
    
    Return
    -----------
    Tuple - height, width, channel 
    
    """
    return image.shape


def extract_lable(FILE_PATH, FILE_NAME, EXTRACT_PATH):
    
    """ This function atracts labels and images from 
        Waymo Database TFRecord file then converts it
        into YOLO format. 
        
        Text files with converted data and images are
        contained into special folder defined by 'extract_path'
        
        Parameters
        ----------
        FILE_PATH - path file from which information is extracted
        FILE_NAME - TFRecord file name
        EXTRACT_PATH - folder path for extracting data
        
        Return
        ----------
        .txt files
        .png images
    
    """
    
    """ Read the TFRecord format file from folder """
    tfr_record_file = tf.data.TFRecordDataset(FILE_PATH+FILE_NAME, compression_type='')

    
    for data in tfr_record_file:
        
        """ Read one frame  """
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        
        
        
        """ This is itarator each image in one frame """
            if len(camera_labels.labels):
                
                UNIQUE_NAME = create_unique_file_name()
                create_txt_file(EXTRACT_PATH, UNIQUE_NAME)
                image = save_image(frame, camera_labels, EXTRACT_PATH, UNIQUE_NAME)
                img_size = get_image_size(image)

                """ This is iterator each camera objects in one image """
                for label in camera_labels.labels:
                    normalize_data = normalize( CLASSES[str(label.type)],
                                                label.box.center_x,
                                                label.box.center_y,
                                                label.box.width,
                                                label.box.length,
                                                img_size
                                              )
                    
                    with open(f'{EXTRACT_PATH}{UNIQUE_NAME}.txt', 'a') as file:
                        file.write(f'{normalize_data}\n')

In [ ]:
def main():
    FOLDER_PATH = ''
    EXTRACT_PATH = ''
    
    for FILENAME in tqdm.tqdm(os.listdir(FOLDER_PATH)):
        if FILENAME.endswith('.tfrecord'):
            extract_lable(FOLDER_PATH, FILENAME, EXTRACT_PATH)
    
if __name__ == '__main__':
    main()    